In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
from pathlib import Path
import json

import numpy as np

home = str(Path.home())

RPLIB_DATA_PREFIX = os.environ.get("RPLIB_DATA_PREFIX")

if RPLIB_DATA_PREFIX is None: # Set default
    RPLIB_DATA_PREFIX=f'{home}/RPLib/data'
    
try:
    import pyrankability as pyrankability
    import pyrplib as pyrplib
except:
    print('Looking for packages in home directory')
    sys.path.insert(0,f"{home}") # Add the home directory relevant paths to the PYTHONPATH
    sys.path.insert(0,f"{home}/ranking_toolbox") # Add the home directory relevant paths to the PYTHONPATH
    sys.path.insert(0,f"{home}/RPLib") # Add the home directory relevant paths to the PYTHONPATH
    import pyrankability
    import pyrplib

import pandas as pd

Looking for packages in home directory


In [47]:
data = pyrplib.data.Data(RPLIB_DATA_PREFIX)

## Unprocessed

In [44]:
data.datasets_df

,Dataset ID,Dataset Name,Description,Type,Loader,Download links
0,1,March Madness 2002,"2002 regular season games, list of teams, and ...",Games,marchmadness.base.Unprocessed,[https://raw.githubusercontent.com/IGARDS/marc...
1,2,March Madness 2003,"2003 regular season games, list of teams, and ...",Games,marchmadness.base.Unprocessed,[https://raw.githubusercontent.com/IGARDS/marc...
2,3,March Madness 2004,"2004 regular season games, list of teams, and ...",Games,marchmadness.base.Unprocessed,[https://raw.githubusercontent.com/IGARDS/marc...
3,4,March Madness 2005,"2005 regular season games, list of teams, and ...",Games,marchmadness.base.Unprocessed,[https://raw.githubusercontent.com/IGARDS/marc...
4,5,March Madness 2006,"2006 regular season games, list of teams, and ...",Games,marchmadness.base.Unprocessed,[https://raw.githubusercontent.com/IGARDS/marc...
...,...,...,...,...,...,...
74,75,March Madness 2018,"2018 regular season games, list of teams, and ...",Games,marchmadness.base.Unprocessed,[https://raw.githubusercontent.com/IGARDS/marc...
75,76,DOM + Noise 10% to 90%,"'{\n ""high"": 5,\n ""low"": 1,\n ""make_u...",Structured Artificial,structured_artificial.base.Unprocessed,[https://raw.githubusercontent.com/IGARDS/stru...
76,77,March Madness 2022,"2022 regular season games, list of teams, and ...",Games,marchmadness.base.Unprocessed,[https://raw.githubusercontent.com/IGARDS/marc...
77,78,Multiple Optimal Solutions,Multiple optimal solutions created to demonstr...,Structured Artificial,structured_artificial.base.Unprocessed,[https://raw.githubusercontent.com/IGARDS/stru...


In [45]:
unprocessed = pyrplib.dataset.load_unprocessed(79,data.datasets_df)

In [46]:
len(unprocessed.data())

150

## Example remove processed

In [38]:
#collection = 'structured_artificial'

In [7]:
#155	76	46	transformers.process_D(data.loc[index,'D'])	D	structured_artificial	{}	14/03/2022 18:22:51

In [39]:
mask = (data.processed_datasets_df['Source Dataset ID'] == unprocessed.dataset_id)

In [40]:
data.processed_datasets_df.drop(data.processed_datasets_df.index[mask],inplace=True)

In [89]:
#data.save_processed_datasets()

## Processed

In [48]:
mask = (data.processed_datasets_df['Source Dataset ID'] == unprocessed.dataset_id)

In [49]:
sum(mask)

150

In [50]:
data.processed_datasets_df[mask].head()

,Dataset ID,Source Dataset ID,Index,Command,Type,Collection,Options,Last Processed Datetime,Identifier,Link
386,687,79,0,"transformers.process_D(data.loc[index,'D'])",D,structured_artificial,{},NaN,NaN,/home/jupyter-pander14/RPLib/data/structured_a...
387,688,79,1,"transformers.process_D(data.loc[index,'D'])",D,structured_artificial,{},NaN,NaN,/home/jupyter-pander14/RPLib/data/structured_a...
388,689,79,2,"transformers.process_D(data.loc[index,'D'])",D,structured_artificial,{},NaN,NaN,/home/jupyter-pander14/RPLib/data/structured_a...
389,690,79,3,"transformers.process_D(data.loc[index,'D'])",D,structured_artificial,{},NaN,NaN,/home/jupyter-pander14/RPLib/data/structured_a...
390,691,79,4,"transformers.process_D(data.loc[index,'D'])",D,structured_artificial,{},NaN,NaN,/home/jupyter-pander14/RPLib/data/structured_a...


In [96]:
# only needed if adding new column data.processed_datasets_df['Identifier'] = ""

In [51]:
get_identifier = lambda item: "Threshold: "+str(item['threshold'])

In [53]:
#index_10 = []
for index in data.processed_datasets_df[mask].index:
    dataset = data.processed_datasets_df.loc[index]
    dataset_id = dataset['Dataset ID']
    processed = pyrplib.dataset.ProcessedD.from_json(f"{RPLIB_DATA_PREFIX}/{collection}/{dataset_id}.json").load()
    unprocessed = pyrplib.dataset.load_unprocessed(processed.source_dataset_id,data.datasets_df)
    item = unprocessed.data().loc[dataset['Index']]
    #if item['percentage'] == 10:
    #    index_10.append(index)
    data.processed_datasets_df.loc[index,'Identifier'] = get_identifier(item)

In [54]:
data.processed_datasets_df[mask]

,Dataset ID,Source Dataset ID,Index,Command,Type,Collection,Options,Last Processed Datetime,Identifier,Link
386,687,79,0,"transformers.process_D(data.loc[index,'D'])",D,structured_artificial,{},NaN,Threshold: 1,/home/jupyter-pander14/RPLib/data/structured_a...
387,688,79,1,"transformers.process_D(data.loc[index,'D'])",D,structured_artificial,{},NaN,Threshold: 1,/home/jupyter-pander14/RPLib/data/structured_a...
388,689,79,2,"transformers.process_D(data.loc[index,'D'])",D,structured_artificial,{},NaN,Threshold: 1,/home/jupyter-pander14/RPLib/data/structured_a...
389,690,79,3,"transformers.process_D(data.loc[index,'D'])",D,structured_artificial,{},NaN,Threshold: 1,/home/jupyter-pander14/RPLib/data/structured_a...
390,691,79,4,"transformers.process_D(data.loc[index,'D'])",D,structured_artificial,{},NaN,Threshold: 1,/home/jupyter-pander14/RPLib/data/structured_a...
...,...,...,...,...,...,...,...,...,...,...
531,832,79,145,"transformers.process_D(data.loc[index,'D'])",D,structured_artificial,{},NaN,Threshold: 5,/home/jupyter-pander14/RPLib/data/structured_a...
532,833,79,146,"transformers.process_D(data.loc[index,'D'])",D,structured_artificial,{},NaN,Threshold: 5,/home/jupyter-pander14/RPLib/data/structured_a...
533,834,79,147,"transformers.process_D(data.loc[index,'D'])",D,structured_artificial,{},NaN,Threshold: 5,/home/jupyter-pander14/RPLib/data/structured_a...
534,835,79,148,"transformers.process_D(data.loc[index,'D'])",D,structured_artificial,{},NaN,Threshold: 5,/home/jupyter-pander14/RPLib/data/structured_a...


In [55]:
data.processed_datasets_df.loc[mask,'Identifier'].value_counts()

Threshold: 5    30
Threshold: 4    30
Threshold: 3    30
Threshold: 2    30
Threshold: 1    30
Name: Identifier, dtype: int64

In [57]:
data.save_processed_datasets()